In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Configurations

In [ ]:
# !pip install trackio

In [ ]:
# # import numpy as np
# import pandas as pd
# import torch
# # from torch import nn
# from torch.utils.data import DataLoader, random_split
# import torchmetrics
# from torchvision import datasets, transforms
# import trackio
# import torch.nn as nn
# from torchvision import models

In [ ]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print("Using device:", device)

In [ ]:
# # Huggingface Access Adding token to environment variable
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
# import os
# os.environ['HF_TOKEN']  = user_secrets.get_secret("hf_token")

In [ ]:
# trackio.init(
#     project="25-t3-nppe1", 
#     space_id="ShreyaAgr/dlgenai-nppe",
#     name="cnn_baseline",
#     group="simple"
# )

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Scratch Model

### Data loading

In [ ]:
# import torch
# from torch.utils.data import Dataset, DataLoader
# from torchvision import transforms
# from PIL import Image
# import pandas as pd
# from pathlib import Path

# # Paths
# DATA_DIR = Path("/kaggle/input/sep-25-dl-gen-ai-nppe-1/face_dataset")
# IMG_DIR = DATA_DIR     # folder with all training images
# CSV_PATH = DATA_DIR / "train.csv" # contains: image_name, gender, age

# # Read CSV
# df = pd.read_csv(CSV_PATH)
# # print("Sample data:\n", df.head())

# # Train-validation split
# from sklearn.model_selection import train_test_split
# train_df, val_df = train_test_split(df, test_size=0.2, stratify=df["gender"], random_state=42)

# # Transforms
# transform = transforms.Compose([
#     transforms.Resize((128, 128)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
# ])

# from torch.utils.data import Dataset
# from PIL import Image
# import torch
# from pathlib import Path

# class FaceDataset(Dataset):
#     def __init__(self, dataframe, root_dir, transform=None):
#         self.data = dataframe
#         self.root_dir = Path(root_dir)
#         self.transform = transform

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         row = self.data.iloc[idx]
#         img_path = self.root_dir / row["full_path"]   #   full path like "face_dataset/train/00001.jpg"
#         image = Image.open(img_path).convert("RGB")

#         gender = torch.tensor(row["gender"], dtype=torch.float32)
#         age = torch.tensor(row["age"], dtype=torch.float32)

#         if self.transform:
#             image = self.transform(image)

#         return image, gender, age


# # Create datasets & loaders
# train_dataset = FaceDataset(train_df, IMG_DIR, transform)
# val_dataset   = FaceDataset(val_df, IMG_DIR, transform)

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
# val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

# # Device setup
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# print(f" Train: {len(train_dataset)} | Val: {len(val_dataset)}")

In [ ]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F

# class AgeGenderCnn(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.features = nn.Sequential(
#             nn.Conv2d(3, 32, 3, padding=1),
#             nn.ReLU(),
#             nn.BatchNorm2d(32),
#             nn.MaxPool2d(2, 2),  # 128 -> 64

#             nn.Conv2d(32, 64, 3, padding=1),
#             nn.ReLU(),
#             nn.BatchNorm2d(64),
#             nn.MaxPool2d(2, 2)   # 64 -> 32
#         )

#         self.flatten = nn.Flatten()
#         self.fc = nn.Sequential(
#             nn.Linear(64 * 32 * 32, 128),  #   fixed here
#             nn.ReLU(),
#             nn.Dropout(0.4),
#             nn.Linear(128, 64),
#             nn.ReLU(),
#             nn.Dropout(0.4)
#         )

#         self.age_head = nn.Linear(64, 1)
#         self.gender_head = nn.Linear(64, 1)

#     def forward(self, x):
#         x = self.features(x)
#         x = self.flatten(x)
#         x = self.fc(x)
#         age = self.age_head(x)
#         gender = torch.sigmoid(self.gender_head(x))
#         return age, gender

In [ ]:
# model = AgeGenderCnn().to(device)

# # Separate losses
# criterion_age = nn.MSELoss()           # regression
# criterion_gender = nn.BCEWithLogitsLoss()  # binary classification

# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# epochs = 3
# loss_history = []

# for epoch in range(epochs):
#     model.train()
#     total_loss = 0

#     for xb, gender_target, age_target in train_loader:  # ensure your dataset returns both
#         xb, age_target, gender_target = xb.to(device), age_target.to(device), gender_target.to(device)

#         pred_age, pred_gender = model(xb)

#         loss_age = criterion_age(pred_age.squeeze(1), age_target.float())
#         loss_gender = criterion_gender(pred_gender.squeeze(1), gender_target.float())
#         loss = loss_age + loss_gender

#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         total_loss += loss.item()

#     avg_loss = total_loss / len(train_loader)
#     # print("iter ",i,"loss ",avg_loss)
#     loss_history.append(avg_loss)

#     print(f"Epoch [{epoch+1}/{epochs}] | Loss: {avg_loss:.4f}")

#     # Log training progress to TrackIO
#     trackio.log({"train/loss": avg_loss, "train/loss_age": loss_age.item(), "train/loss_gender": loss_gender.item(), "epoch": epoch})

# trackio.finish()
# print(" Training complete!")

In [ ]:
# #kaggleHub
# import kagglehub
# KAGGLE_USERNAME = "shreyaaggarwal09"  # replace
# MODEL = "age-gender-cnn"
# FRAMEWORK = "pytorch"
# VARIATION = "baseline"
# handle = f"{KAGGLE_USERNAME}/{MODEL}/{FRAMEWORK}/{VARIATION}"

# kagglehub.model_upload(handle, model_path, version_notes="Initial baseline model trained with TrackIO logging")

In [ ]:
# import matplotlib.pyplot as plt

# plt.figure(figsize=(8,5))
# plt.plot(range(1, epochs+1), loss_history, marker='o', color='teal')
# plt.title("Training Loss Curve")
# plt.xlabel("Epoch")
# plt.ylabel("Loss")
# plt.grid(True)
# plt.show()

In [ ]:
# import torch
# from torch.utils.data import Dataset, DataLoader
# from torchvision import transforms
# from PIL import Image
# import pandas as pd
# from pathlib import Path

# # Path to trained model
# model_path = '/kaggle/input/age-gender-cnn/pytorch/baseline/1/age_gender_model.pth'

# # Device
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Define your model class exactly as during training
# # Make sure AgeGenderCnn class is defined above this cell
# model = AgeGenderCnn().to(device)
# model.load_state_dict(torch.load(model_path, map_location=device))
# model.eval()

# # Load test CSV
# test_csv_path = "/kaggle/input/sep-25-dl-gen-ai-nppe-1/face_dataset/test.csv"
# test_df = pd.read_csv(test_csv_path)

# # Folder where test images are stored
# IMG_DIR = Path("/kaggle/input/sep-25-dl-gen-ai-nppe-1/face_dataset/test")  

# # Image transforms (same as training)
# transform = transforms.Compose([
#     transforms.Resize((128, 128)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])
# ])

# # Dataset for inference
# class TestDataset(Dataset):
#     def __init__(self, df, root_dir, transform=None):
#         self.df = df
#         self.root_dir = root_dir
#         self.transform = transform

#     def __len__(self):
#         return len(self.df)

#     def __getitem__(self, idx):
#         # Convert numeric ID to filename (zero-padded)
#         img_id = self.df.iloc[idx]['id']            # e.g., 0, 1, 2
#         img_name = f"{int(img_id):05d}.jpg"         # zero-padded to 5 digits
#         img_path = self.root_dir / img_name

#         image = Image.open(img_path).convert("RGB")
#         if self.transform:
#             image = self.transform(image)
#         return image, idx

# # DataLoader
# test_dataset = TestDataset(test_df, root_dir=IMG_DIR, transform=transform)
# test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# # Inference loop
# ids = []
# pred_ages = []
# pred_genders = []

# with torch.no_grad():
#     for xb, idx_batch in test_loader:
#         xb = xb.to(device)
#         pred_age, pred_gender = model(xb)

#         # Convert idx_batch tensor to list of ints
#         idx_list = idx_batch.tolist()
#         ids.extend([test_df.iloc[i]['id'] for i in idx_list])

#         pred_ages.extend(pred_age.squeeze(1).cpu().numpy())
#         pred_genders.extend((pred_gender.squeeze(1) > 0.5).long().cpu().numpy())

# # Create submission DataFrame
# submission_df = pd.DataFrame({
#     'id': ids,
#     'gender': pred_genders,
#     'age': pred_ages
# })
# submission_df['age'] = submission_df['age'].round().astype(int)

# submission_path = '/kaggle/working/submission.csv'
# submission_df.to_csv(submission_path, index=False)
# print("  Submission file created successfully!")
# print(submission_df.head())

# Fine tuned model

### EfficientNet

In [ ]:
# #fine tuned
# import torch
# from torch.utils.data import Dataset, DataLoader
# from torchvision import transforms
# from PIL import Image
# import pandas as pd
# from pathlib import Path

# # Paths
# DATA_DIR = Path("/kaggle/input/sep-25-dl-gen-ai-nppe-1/face_dataset")
# IMG_DIR = DATA_DIR     # folder with all training images
# CSV_PATH = DATA_DIR / "train.csv" # contains: image_name, gender, age

# # Read CSV
# df = pd.read_csv(CSV_PATH)
# # Compute mean and std for age normalization
# age_mean = df["age"].mean()
# age_std = df["age"].std()

# print(f"Age mean: {age_mean:.2f}, std: {age_std:.2f}")

# # print("Sample data:\n", df.head())

# # Train-validation split
# from sklearn.model_selection import train_test_split
# train_df, val_df = train_test_split(df, test_size=0.2, stratify=df["gender"], random_state=42)

# #finetuned
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                          std=[0.229, 0.224, 0.225])
# ])

# from torch.utils.data import Dataset
# from PIL import Image
# import torch
# from pathlib import Path

# class FaceDataset(Dataset):
#     def __init__(self, dataframe, root_dir, transform=None, age_mean=None, age_std=None):
#         self.data = dataframe
#         self.root_dir = Path(root_dir)
#         self.transform = transform
#         self.age_mean = age_mean
#         self.age_std = age_std

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         row = self.data.iloc[idx]
#         img_path = self.root_dir / row["full_path"]
#         image = Image.open(img_path).convert("RGB")

#         gender = torch.tensor(row["gender"], dtype=torch.float32)
#         age = torch.tensor(row["age"], dtype=torch.float32)

#         #   Normalize age
#         if self.age_mean is not None and self.age_std is not None:
#             age = (age - self.age_mean) / self.age_std

#         if self.transform:
#             image = self.transform(image)

#         return image, gender, age

# # Create datasets & loaders
# train_dataset = FaceDataset(train_df, IMG_DIR, transform, age_mean, age_std)
# val_dataset   = FaceDataset(val_df, IMG_DIR, transform, age_mean, age_std)

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
# val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

# # Device setup
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# print(f" Train: {len(train_dataset)} | Val: {len(val_dataset)}")

In [ ]:
# import torch
# from torchvision import transforms
# from PIL import Image
# import pandas as pd
# from pathlib import Path
# from torch.utils.data import Dataset, DataLoader

# # Load pretrained model class (must match your training model code)
# class AgeGenderEfficientNet(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.backbone = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)

#         for param in self.backbone.parameters():
#             param.requires_grad = False

#         in_features = self.backbone.classifier[1].in_features
#         self.backbone.classifier = nn.Identity()

#         self.gender_head = nn.Linear(in_features, 1)
#         self.age_head = nn.Linear(in_features, 1)

#     def forward(self, x):
#         feats = self.backbone(x)
#         gender = torch.sigmoid(self.gender_head(feats))
#         age = self.age_head(feats)
#         return age, gender 


In [ ]:
# model = AgeGenderEfficientNet().to(device)
# criterion_gender = nn.BCELoss()      # binary gender
# criterion_age = nn.MSELoss()         # regression age
# optimizer = torch.optim.Adam(
#     list(model.gender_head.parameters()) + list(model.age_head.parameters()),
#     lr=1e-3
# )

In [ ]:
# loss_history = []

# for epoch in range(10):

#     # Unfreeze backbone after 5 epochs (optional fine-tuning)
#     if epoch == 5:
#         print("Unfreezing last block for fine-tuning...")
#         for param in model.backbone.features[-1].parameters():
#             param.requires_grad = True
#         optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

#     model.train()
#     running_loss = 0.0
#     running_age_loss = 0.0
#     running_gender_loss = 0.0

#     for imgs, gender, age in train_loader:
#         imgs = imgs.to(device)
#         gender = gender.to(device).float()
#         age = age.to(device).float()

#         pred_age, pred_gender = model(imgs)

#         pred_gender = pred_gender.squeeze(1)
#         pred_age = pred_age.squeeze(1)

#         loss_gender = criterion_gender(pred_gender, gender)
#         loss_age = criterion_age(pred_age, age)

#         loss = loss_gender + loss_age

#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()
#         running_age_loss += loss_age.item()
#         running_gender_loss += loss_gender.item()

#     avg_loss = running_loss / len(train_loader)
#     avg_age_loss = running_age_loss / len(train_loader)
#     avg_gender_loss = running_gender_loss / len(train_loader)

#     loss_history.append(avg_loss)

#     print(f"Epoch [{epoch+1}/10] | Total Loss: {avg_loss:.4f} | Age Loss: {avg_age_loss:.4f} | Gender Loss: {avg_gender_loss:.4f}")

#     # TrackIO logging (safe and meaningful now)
#     trackio.log({
#         "train/loss": avg_loss,
#         "train/loss_age": avg_age_loss,
#         "train/loss_gender": avg_gender_loss,
#         "epoch": epoch
#     })
# trackio.finish()
# print(" Training complete!")

In [ ]:
# model_path = Path("trained_model")
# model_path.mkdir(exist_ok=True)
# torch.save(model.state_dict(), model_path / "age_gender_model.pth")

In [ ]:
# ##pretrained
# import matplotlib.pyplot as plt

# plt.figure(figsize=(8, 5))
# plt.plot(range(1, len(loss_history) + 1), loss_history, marker='o', color='teal', linewidth=2)
# plt.title("Training Loss Curve", fontsize=14)
# plt.xlabel("Epoch", fontsize=12)
# plt.ylabel("Total Loss", fontsize=12)
# plt.grid(True, linestyle='--', alpha=0.6)
# plt.tight_layout()
# plt.show()

In [ ]:
# #kaggleHub
# import kagglehub
# KAGGLE_USERNAME = "shreyaaggarwal09"  # replace
# MODEL = "age-gender-pre-trained-cnn"
# FRAMEWORK = "pytorch"
# VARIATION = "baseline"
# handle = f"{KAGGLE_USERNAME}/{MODEL}/{FRAMEWORK}/{VARIATION}"

# kagglehub.model_upload(handle, model_path, version_notes="Initial baseline model trained with TrackIO logging")

In [ ]:
# import torch
# import torch.nn as nn
# from torchvision import transforms, models
# from PIL import Image
# import pandas as pd
# from pathlib import Path
# from torch.utils.data import Dataset, DataLoader

# # ------------------------------
# # Model (must match training)
# # ------------------------------
# class AgeGenderEfficientNet(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.backbone = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)

#         for param in self.backbone.parameters():
#             param.requires_grad = False

#         in_features = self.backbone.classifier[1].in_features
#         self.backbone.classifier = nn.Identity()

#         self.gender_head = nn.Linear(in_features, 1)
#         self.age_head = nn.Linear(in_features, 1)

#     def forward(self, x):
#         feats = self.backbone(x)
#         gender = torch.sigmoid(self.gender_head(feats))
#         age = self.age_head(feats)
#         return age, gender


# # ------------------------------
# # Load Model
# # ------------------------------
# model_path = "/kaggle/input/age-gender-pre-trained-cnn/pytorch/baseline/1/age_gender_model.pth"
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# model = AgeGenderEfficientNet().to(device)

# checkpoint = torch.load(model_path, map_location=device)
# state_dict = model.state_dict()

# # Load only matching layers
# filtered = {k: v for k, v in checkpoint.items() if k in state_dict and v.shape == state_dict[k].shape}
# state_dict.update(filtered)
# model.load_state_dict(state_dict, strict=False)

# model.eval()
# print(f"  Loaded weights for {len(filtered)}/{len(state_dict)} layers")

# # ------------------------------
# # Load Test CSV
# # ------------------------------
# test_csv_path = "/kaggle/input/sep-25-dl-gen-ai-nppe-1/face_dataset/test.csv"
# test_df = pd.read_csv(test_csv_path)

# IMG_DIR = Path("/kaggle/input/sep-25-dl-gen-ai-nppe-1/face_dataset/test")

# # ------------------------------
# # IMPORTANT: Use SAME normalization as TRAINING
# # ------------------------------
# transform = transforms.Compose([
#     transforms.Resize((128, 128)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]) 
# ])

# # ------------------------------
# # Dataset
# # ------------------------------
# class TestDataset(Dataset):
#     def __init__(self, df, root, transform):
#         self.df = df
#         self.root = root
#         self.transform = transform

#     def __len__(self):
#         return len(self.df)

#     def __getitem__(self, idx):
#         img_id = int(self.df.iloc[idx]["id"])
#         img_name = f"{img_id:05d}.jpg"
#         img_path = self.root / img_name

#         image = Image.open(img_path).convert("RGB")
#         image = self.transform(image)

#         return image, idx

# test_dataset = TestDataset(test_df, IMG_DIR, transform)
# test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# # ------------------------------
# # Inference
# # ------------------------------
# ids, pred_genders, pred_ages = [], [], []

# with torch.no_grad():
#     for xb, idxs in test_loader:
#         xb = xb.to(device)

#         age_out, gender_out = model(xb)

#         ages = age_out.squeeze(1).cpu().numpy()
#         genders = (gender_out.squeeze(1) > 0.5).long().cpu().numpy()

#         for b, df_index in enumerate(idxs.tolist()):
#             ids.append(test_df.iloc[df_index]["id"])
#             pred_ages.append(ages[b])
#             pred_genders.append(genders[b])

# # ------------------------------
# # Create & Save Submission
# # ------------------------------
# submission = pd.DataFrame({
#     "id": ids,
#     "gender": pred_genders,
#     "age": pred_ages
# })

# submission["age"] = submission["age"].round().astype(int)

# out_path = "/kaggle/working/submission.csv"
# submission.to_csv(out_path, index=False)

# print("  Submission ready:", out_path)
# print(submission.head())

## ConvNext Tiny 1K 224 (Final Model)

In [1]:
!pip install trackio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.0/875.0 kB 17.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 MB 28.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.4/325.4 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 97.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.8/444.8 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 74.5 MB/s eta 0:00:00
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.41.5
    Uninstalling pydantic_core-2.41.5:
      Successfully uninstalled pydantic_core-2.41.5
  Attempting uninstall: plotly
    Found existing installation: plotly 5.24.1
    Uninstalling plotly-5.24.1:
      Successfully uninstalled plotly-5.24.1
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.12

In [2]:
# Huggingface Access Adding token to environment variable
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
import os
os.environ['HF_TOKEN']  = user_secrets.get_secret("hf_token")

In [3]:
import trackio
import torch
trackio.init(
    project="25-t3-nppe1", 
    space_id="ShreyaAgr/dlgenai-nppe",
    name="convexnet12",
    group="simple"
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

* Trackio project initialized: 25-t3-nppe1
* Trackio metrics will be synced to Hugging Face Dataset: ShreyaAgr/dlgenai-nppe-dataset
* Found existing space: https://huggingface.co/spaces/ShreyaAgr/dlgenai-nppe
* View dashboard by going to: https://ShreyaAgr-dlgenai-nppe.hf.space/


* Created new run: convexnet12


In [4]:
# ===========================================
#  TRAINING SCRIPT  (ConvNeXtV2-Tiny Optimized, No Age Normalization)
# ===========================================

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import ConvNextV2Model, AutoImageProcessor
from PIL import Image
from pathlib import Path
import pandas as pd
from tqdm import tqdm

# ------------------------------
# Config
# ------------------------------
MODEL_NAME = "facebook/convnextv2-tiny-1k-224"
BATCH_SIZE = 16
EPOCHS = 12
LR = 1e-4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

TRAIN_CSV = "/kaggle/input/sep-25-dl-gen-ai-nppe-1/face_dataset/train.csv"
IMG_DIR = Path("/kaggle/input/sep-25-dl-gen-ai-nppe-1/face_dataset/train")

# ------------------------------
# Preprocessing
# ------------------------------
processor = AutoImageProcessor.from_pretrained(MODEL_NAME)
train_df = pd.read_csv(TRAIN_CSV)

# ------------------------------
# Dataset
# ------------------------------
class FaceDataset(Dataset):
    def __init__(self, df, root, processor):
        self.df = df
        self.root = root
        self.processor = processor

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = self.root / f"{int(row['id']):05d}.jpg"
        image = Image.open(img_path).convert("RGB")
        pixel_values = self.processor(images=image, return_tensors="pt")["pixel_values"].squeeze(0)

        age = torch.tensor(row["age"], dtype=torch.float32)
        gender = torch.tensor(row["gender"], dtype=torch.float32)
        return pixel_values, age, gender


train_loader = DataLoader(
    FaceDataset(train_df, IMG_DIR, processor),
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2
)

# ------------------------------
# Model
# ------------------------------
class AgeGenderConvNeXt(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = ConvNextV2Model.from_pretrained(MODEL_NAME)
        in_features = self.backbone.config.hidden_sizes[-1]

        # Freeze backbone initially
        for p in self.backbone.parameters():
            p.requires_grad = False

        self.gender_head = nn.Linear(in_features, 1)
        self.age_head = nn.Linear(in_features, 1)

    def forward(self, x):
        feats = self.backbone(x).pooler_output
        gender = torch.sigmoid(self.gender_head(feats))
        age = self.age_head(feats)  # direct regression output (no sigmoid)
        return age, gender


model = AgeGenderConvNeXt().to(DEVICE)

# ------------------------------
# Loss & Optimizer
# ------------------------------
criterion_age = nn.SmoothL1Loss()  # regression
criterion_gender = nn.BCELoss()    # binary classification

optimizer = torch.optim.Adam(
    list(model.gender_head.parameters()) + list(model.age_head.parameters()),
    lr=LR
)

# ------------------------------
# Training Loop
# ------------------------------
for epoch in range(EPOCHS):
    #   Gradual unfreezing after 2 epochs
    if epoch == 2:
        for p in model.backbone.encoder.stages[-1].parameters():
            p.requires_grad = True
        optimizer = torch.optim.Adam(model.parameters(), lr=LR / 10)
        print("Unfroze last ConvNeXt block for fine-tuning")

    model.train()
    total_age_loss, total_gender_loss = 0.0, 0.0

    for xb, age, gender in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        xb, age, gender = xb.to(DEVICE), age.to(DEVICE).unsqueeze(1), gender.to(DEVICE).unsqueeze(1)

        optimizer.zero_grad()
        pred_age, pred_gender = model(xb)

        loss_age = criterion_age(pred_age, age)
        loss_gender = criterion_gender(pred_gender, gender)

        # Weighted total loss
        loss = 0.1 * loss_age + loss_gender

        loss.backward()
        optimizer.step()

        total_age_loss += loss_age.item()
        total_gender_loss += loss_gender.item()

    avg_age_loss = total_age_loss / len(train_loader)
    avg_gender_loss = total_gender_loss / len(train_loader)
    avg_total_loss = 0.1 * avg_age_loss + avg_gender_loss

    print(f"Epoch [{epoch+1}/{EPOCHS}] | "
          f"Age Loss: {avg_age_loss:.4f} | "
          f"Gender Loss: {avg_gender_loss:.4f} | "
          f"Total Loss: {avg_total_loss:.4f}")

#Log training progress to TrackIO
    trackio.log({"train/loss": avg_total_loss, "train/loss_age": avg_age_loss, "train/loss_gender": avg_gender_loss, "epoch": epoch})
trackio.finish()

# ------------------------------
# Save Model
# ------------------------------
Path("trained_model").mkdir(exist_ok=True)
torch.save(model.state_dict(), "trained_model/age_gender_convnext.pth")
print("  Model saved at trained_model/age_gender_convnext.pth")

2025-11-11 12:37:51.356886: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762864671.749866      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762864671.889787      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

Epoch [10/12] | Age Loss: 4.5661 | Gender Loss: 0.0901 | Total Loss: 0.5467


Epoch 11/12: 100%|██████████| 2170/2170 [04:31<00:00,  7.99it/s]


Epoch [11/12] | Age Loss: 4.2440 | Gender Loss: 0.0782 | Total Loss: 0.5026


Epoch 12/12: 100%|██████████| 2170/2170 [04:31<00:00,  7.98it/s]


Epoch [12/12] | Age Loss: 3.9227 | Gender Loss: 0.0672 | Total Loss: 0.4595
* Run finished. Uploading logs to Trackio (please wait...)
  Model saved at trained_model/age_gender_convnext.pth


In [ ]:
# # ===========================================
# #  Upload trained PyTorch model to KaggleHub
# # ===========================================

# import kagglehub

# # Define handle details
# KAGGLE_USERNAME = "shreyaaggarwal09"     # your Kaggle username
# MODEL = "age-12gender_wn-con-trained-model"   # model repository name
# FRAMEWORK = "pytorch"
# VARIATION = "age_gender_convnext"
# handle = f"{KAGGLE_USERNAME}/{MODEL}/{FRAMEWORK}/{VARIATION}"

# # Path to your trained model (.pth file)
# model_path = "trained_model/age_gender_convnext.pth"  # <-- same as your training script

# # Upload model
# kagglehub.model_upload(
#     handle,
#     model_path,
#     version_notes="Initial baseline model trained with TrackIO logging"
# )

# print("  Model uploaded successfully to KaggleHub!")

In [1]:
#inference

import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import ConvNextV2Model, AutoImageProcessor
from PIL import Image
from tqdm import tqdm
import pandas as pd
from pathlib import Path
from kaggle_secrets import UserSecretsClient


# Hugging Face Authentication
user_secrets = UserSecretsClient()
os.environ['HF_TOKEN'] = user_secrets.get_secret("hf_token")
print(" Hugging Face token loaded successfully.")

# Config
MODEL_NAME = "facebook/convnextv2-tiny-1k-224"
MODEL_PATH = "/kaggle/input/age-12gender_wn-con-trained-model/pytorch/age_gender_convnext/1/age_gender_convnext.pth"
TEST_CSV = "/kaggle/input/sep-25-dl-gen-ai-nppe-1/face_dataset/test.csv"
IMG_DIR = Path("/kaggle/input/sep-25-dl-gen-ai-nppe-1/face_dataset/test")
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Model Definition (Same as Training)

class AgeGenderConvNeXt(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = ConvNextV2Model.from_pretrained(MODEL_NAME, token=os.environ["HF_TOKEN"])
        in_features = self.backbone.config.hidden_sizes[-1]

        # Freeze backbone during inference
        for p in self.backbone.parameters():
            p.requires_grad = False

        # Output heads
        self.gender_head = nn.Linear(in_features, 1)
        self.age_head = nn.Linear(in_features, 1)  # direct regression (no sigmoid)

    def forward(self, x):
        feats = self.backbone(x).pooler_output
        gender = torch.sigmoid(self.gender_head(feats))  # binary classification
        age = self.age_head(feats)                       # raw regression output
        return age, gender

# Load Trained Model Weights
print(" Loading model weights...")
model = AgeGenderConvNeXt().to(DEVICE)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.eval()
print(f"  Model loaded successfully on {DEVICE}")


# Dataset + DataLoader
processor = AutoImageProcessor.from_pretrained(MODEL_NAME, token=os.environ["HF_TOKEN"])

class TestDataset(Dataset):
    def __init__(self, df, root, processor):
        self.df = df
        self.root = root
        self.processor = processor

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_id = int(self.df.iloc[idx]["id"])
        img_path = self.root / f"{img_id:05d}.jpg"
        image = Image.open(img_path).convert("RGB")
        pixel_values = self.processor(images=image, return_tensors="pt")["pixel_values"].squeeze(0)
        return pixel_values, idx

#   Load test data
test_df = pd.read_csv(TEST_CSV)
test_loader = DataLoader(
    TestDataset(test_df, IMG_DIR, processor),
    batch_size=64,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

torch.backends.cudnn.benchmark = True  # for speed optimization


# Inference with tqdm Progress
ids, pred_ages, pred_genders = [], [], []

print("⚡ Starting inference...")
with torch.no_grad():
    for xb, idxs in tqdm(test_loader, total=len(test_loader), desc="🧠 Predicting"):
        xb = xb.to(DEVICE, non_blocking=True)
        with torch.cuda.amp.autocast():  # mixed precision for faster inference
            age_out, gender_out = model(xb)

        # Post-processing
        pred_age = age_out.squeeze(1).cpu().numpy()
        pred_age = pred_age.clip(0, 100)  # limit to valid age range
        pred_gender = (gender_out.squeeze(1) > 0.5).long().cpu().numpy()

        for i, df_idx in enumerate(idxs.tolist()):
            ids.append(test_df.iloc[df_idx]["id"])
            pred_ages.append(pred_age[i])
            pred_genders.append(pred_gender[i])

        if len(ids) % 500 < len(xb):
            print(f"Processed {len(ids)} / {len(test_df)} images...")


# Save Submission
submission = pd.DataFrame({
    "id": ids,
    "gender": pred_genders,
    "age": [int(round(a)) for a in pred_ages]
})

out_path = "/kaggle/working/submission.csv"
submission.to_csv(out_path, index=False)

print(f"  Submission saved to: {out_path}")
print(submission.head())

2025-11-10 14:36:01.819236: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762785362.012645      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762785362.065811      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

Processed 8000 / 8677 images...


🧠 Predicting:  98%|█████████▊| 133/136 [00:32<00:00,  4.47it/s]

Processed 8512 / 8677 images...


🧠 Predicting: 100%|██████████| 136/136 [00:32<00:00,  4.13it/s]

  Submission saved to: /kaggle/working/submission.csv
   id  gender  age
0   0       1   39
1   1       0   34
2   2       1   25
3   3       1   31
4   4       1   52


In [2]:
import os
print(os.listdir('/kaggle/working/'))

['submission.csv', '.virtual_documents']


### Resnet50

In [ ]:
# !pip install trackio

In [ ]:
# # Huggingface Access Adding token to environment variable
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
# import os
# os.environ['HF_TOKEN']  = user_secrets.get_secret("hf_token")

In [ ]:
# import trackio
# import torch
# trackio.init(
#     project="25-t3-nppe1", 
#     space_id="ShreyaAgr/dlgenai-nppe",
#     name="convexnet12",
#     group="cnn_baseline"
# )

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# # ===========================================
# #  TRAINING SCRIPT  (ResNet-50 Optimized + Fixed)
# # ===========================================

# import torch
# import torch.nn as nn
# from torch.utils.data import Dataset, DataLoader
# from transformers import AutoModel, AutoImageProcessor
# from PIL import Image
# from pathlib import Path
# import pandas as pd
# from tqdm import tqdm

# # ------------------------------
# # Config
# # ------------------------------
# MODEL_NAME = "microsoft/resnet-50"
# BATCH_SIZE = 16
# EPOCHS = 10
# LR = 1e-4
# DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# TRAIN_CSV = "/kaggle/input/sep-25-dl-gen-ai-nppe-1/face_dataset/train.csv"
# IMG_DIR = Path("/kaggle/input/sep-25-dl-gen-ai-nppe-1/face_dataset/train")

# # ------------------------------
# # Preprocessing
# # ------------------------------
# processor = AutoImageProcessor.from_pretrained(MODEL_NAME)
# train_df = pd.read_csv(TRAIN_CSV)

# #   Normalize age targets
# mean_age = train_df["age"].mean()
# std_age = train_df["age"].std()
# train_df["age_norm"] = (train_df["age"] - mean_age) / std_age

# # ------------------------------
# # Dataset
# # ------------------------------
# class FaceDataset(Dataset):
#     def __init__(self, df, root, processor):
#         self.df = df
#         self.root = root
#         self.processor = processor

#     def __len__(self):
#         return len(self.df)

#     def __getitem__(self, idx):
#         row = self.df.iloc[idx]
#         img_path = self.root / f"{int(row['id']):05d}.jpg"
#         image = Image.open(img_path).convert("RGB")
#         pixel_values = self.processor(images=image, return_tensors="pt")["pixel_values"].squeeze(0)

#         age = torch.tensor(row["age_norm"], dtype=torch.float32)
#         gender = torch.tensor(row["gender"], dtype=torch.float32)
#         return pixel_values, age, gender


# train_loader = DataLoader(
#     FaceDataset(train_df, IMG_DIR, processor),
#     batch_size=BATCH_SIZE,
#     shuffle=True,
#     num_workers=2
# )

# # ------------------------------
# # Model
# # ------------------------------
# class AgeGenderResNet(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.backbone = AutoModel.from_pretrained(MODEL_NAME)

#         # figure out backbone output size
#         if hasattr(self.backbone.config, "hidden_sizes"):
#             in_features = self.backbone.config.hidden_sizes[-1]
#         elif hasattr(self.backbone.config, "hidden_size"):
#             in_features = self.backbone.config.hidden_size
#         else:
#             in_features = 2048  # Default for ResNet-50

#         # Freeze backbone initially
#         for p in self.backbone.parameters():
#             p.requires_grad = False

#         # Two heads
#         self.gender_head = nn.Linear(in_features, 1)
#         self.age_head = nn.Linear(in_features, 1)

#     def forward(self, x):
#         outputs = self.backbone(x)
#         if hasattr(outputs, "last_hidden_state"):
#             feats = outputs.last_hidden_state
#         elif hasattr(outputs, "pooler_output"):
#             feats = outputs.pooler_output
#         elif isinstance(outputs, torch.Tensor):
#             feats = outputs
#         else:
#             feats = outputs[0]

#         # ResNet gives 4D (B, 2048, 7, 7)
#         if feats.ndim == 4:
#             feats = feats.mean(dim=[2, 3])  # global average pool -> (B, 2048)

#         gender = torch.sigmoid(self.gender_head(feats))
#         age = torch.sigmoid(self.age_head(feats)) * 100
#         return age, gender


# model = AgeGenderResNet().to(DEVICE)

# # ------------------------------
# # Loss & Optimizer
# # ------------------------------
# criterion_age = nn.SmoothL1Loss()
# criterion_gender = nn.BCELoss()

# optimizer = torch.optim.Adam(
#     list(model.gender_head.parameters()) + list(model.age_head.parameters()),
#     lr=LR
# )

# # ------------------------------
# # Training Loop
# # ------------------------------
# for epoch in range(EPOCHS):
 
#     model.train()
#     total_age_loss, total_gender_loss = 0.0, 0.0

#     for xb, age, gender in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
#         xb, age, gender = xb.to(DEVICE), age.to(DEVICE), gender.to(DEVICE).unsqueeze(1)

#         optimizer.zero_grad()
#         pred_age, pred_gender = model(xb)

#         pred_age_norm = (pred_age - mean_age) / std_age
#         loss_age = criterion_age(pred_age_norm, age)
#         loss_gender = criterion_gender(pred_gender, gender)

#         loss = 0.1 * loss_age + loss_gender
#         loss.backward()
#         optimizer.step()

#         total_age_loss += loss_age.item()
#         total_gender_loss += loss_gender.item()

#     avg_age_loss = total_age_loss / len(train_loader)
#     avg_gender_loss = total_gender_loss / len(train_loader)
#     avg_total_loss = 0.1 * avg_age_loss + avg_gender_loss

#     print(f"Epoch [{epoch+1}/{EPOCHS}] | "
#           f"Age Loss: {avg_age_loss:.4f} | "
#           f"Gender Loss: {avg_gender_loss:.4f} | "
#           f"Total Loss: {avg_total_loss:.4f}")

# # ------------------------------
# # Save Model
# # ------------------------------
# Path("trained_model").mkdir(exist_ok=True)
# torch.save(model.state_dict(), "trained_model/age_gender_resnet50.pth")
# print("  Model saved at trained_model/age_gender_resnet50.pth")

In [ ]:
# # ===========================================
# #  Upload trained PyTorch model to KaggleHub
# # ===========================================

# import kagglehub

# #   Define handle details
# KAGGLE_USERNAME = "shreyaaggarwal09"     # your Kaggle username
# MODEL = "resnet50-v2age-gender"   # model repository name
# FRAMEWORK = "pytorch"
# VARIATION = "baseline"
# handle = f"{KAGGLE_USERNAME}/{MODEL}/{FRAMEWORK}/{VARIATION}"

# #   Path to your trained model (.pth file)
# model_path = "trained_model/age_gender_resnet50.pth"  # <-- same as your training script

# #   Upload model
# kagglehub.model_upload(
#     handle,
#     model_path,
#     version_notes="Initial baseline model trained with TrackIO logging"
# )

# print("  Model uploaded successfully to KaggleHub!")

In [ ]:
# # ===========================================
# # 🚀 FINAL INFERENCE (ResNet-50 Optimized)
# # ===========================================

# import os
# import torch
# import torch.nn as nn
# from torch.utils.data import Dataset, DataLoader
# from transformers import AutoModel, AutoImageProcessor
# from PIL import Image
# from tqdm import tqdm
# import pandas as pd
# from pathlib import Path
# from kaggle_secrets import UserSecretsClient

# # ------------------------------
# #   Hugging Face Token
# # ------------------------------
# user_secrets = UserSecretsClient()
# os.environ['HF_TOKEN'] = user_secrets.get_secret("hf_token")
# print(" Hugging Face token loaded successfully.")

# # ------------------------------
# #   Config
# # ------------------------------
# MODEL_NAME = "microsoft/resnet-50"
# MODEL_PATH = "/kaggle/input/resnet50-v2age-gender/pytorch/baseline/1/age_gender_resnet50.pth"
# TRAIN_CSV = "/kaggle/input/sep-25-dl-gen-ai-nppe-1/face_dataset/train.csv"
# TEST_CSV = "/kaggle/input/sep-25-dl-gen-ai-nppe-1/face_dataset/test.csv"
# IMG_DIR = Path("/kaggle/input/sep-25-dl-gen-ai-nppe-1/face_dataset/test")
# DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # ------------------------------
# #   Load Normalization Info (mean, std from training)
# # ------------------------------
# train_df = pd.read_csv(TRAIN_CSV)
# mean_age = train_df["age"].mean()
# std_age = train_df["age"].std()
# print(f"  Mean Age: {mean_age:.2f} | Std Age: {std_age:.2f}")

# # ------------------------------
# #   Model Definition (same architecture as training)
# # ------------------------------
# class AgeGenderResNet(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.backbone = AutoModel.from_pretrained(MODEL_NAME, token=os.environ["HF_TOKEN"])

#         # Determine output size
#         if hasattr(self.backbone.config, "hidden_sizes"):
#             in_features = self.backbone.config.hidden_sizes[-1]
#         elif hasattr(self.backbone.config, "hidden_size"):
#             in_features = self.backbone.config.hidden_size
#         else:
#             in_features = 2048  # for ResNet-50

#         # Two output heads
#         self.gender_head = nn.Linear(in_features, 1)
#         self.age_head = nn.Linear(in_features, 1)

#     def forward(self, x):
#         outputs = self.backbone(x)

#         # Handle different backbone output formats
#         if hasattr(outputs, "last_hidden_state"):
#             feats = outputs.last_hidden_state
#         elif hasattr(outputs, "pooler_output"):
#             feats = outputs.pooler_output
#         elif isinstance(outputs, torch.Tensor):
#             feats = outputs
#         else:
#             feats = outputs[0]

#         if feats.ndim == 4:
#             feats = feats.mean(dim=[2, 3])  # GAP for ResNet

#         gender = torch.sigmoid(self.gender_head(feats))
#         age = torch.sigmoid(self.age_head(feats)) * 100  # scaled 0–100
#         return age, gender


# # ------------------------------
# # Load Trained Weights
# # ------------------------------
# print("Loading trained model weights...")
# model = AgeGenderResNet().to(DEVICE)
# state_dict = torch.load(MODEL_PATH, map_location=DEVICE)
# model.load_state_dict(state_dict, strict=False)
# model.eval()
# print(f" Model loaded successfully on {DEVICE}")

# # ------------------------------
# # Dataset + DataLoader
# # ------------------------------
# processor = AutoImageProcessor.from_pretrained(MODEL_NAME, token=os.environ["HF_TOKEN"])

# class TestDataset(Dataset):
#     def __init__(self, df, root, processor):
#         self.df = df
#         self.root = root
#         self.processor = processor

#     def __len__(self):
#         return len(self.df)

#     def __getitem__(self, idx):
#         img_id = int(self.df.iloc[idx]["id"])
#         img_path = self.root / f"{img_id:05d}.jpg"
#         image = Image.open(img_path).convert("RGB")
#         pixel_values = self.processor(images=image, return_tensors="pt")["pixel_values"].squeeze(0)
#         return pixel_values, idx


# test_df = pd.read_csv(TEST_CSV)
# test_loader = DataLoader(
#     TestDataset(test_df, IMG_DIR, processor),
#     batch_size=64,
#     shuffle=False,
#     num_workers=2,
#     pin_memory=True
# )

# torch.backends.cudnn.benchmark = True  # speed optimization

# # ------------------------------
# # Inference Loop
# # ------------------------------
# ids, pred_ages, pred_genders = [], [], []
# print("Starting inference...")

# with torch.no_grad():
#     for xb, idxs in tqdm(test_loader, total=len(test_loader), desc=" Predicting"):
#         xb = xb.to(DEVICE, non_blocking=True)

#         with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
#             age_out, gender_out = model(xb)

#         pred_age = age_out.squeeze(1).cpu().numpy().clip(0, 100)
#         pred_gender = (gender_out.squeeze(1) > 0.5).long().cpu().numpy()

#         for i, df_idx in enumerate(idxs.tolist()):
#             ids.append(test_df.iloc[df_idx]["id"])
#             pred_ages.append(int(round(pred_age[i])))
#             pred_genders.append(pred_gender[i])

# # ------------------------------
# # Create Submission
# # ------------------------------
# submission = pd.DataFrame({
#     "id": ids,
#     "gender": pred_genders,
#     "age": pred_ages
# })

# out_path = "/kaggle/working/submission.csv"
# submission.to_csv(out_path, index=False)

# print(f"\n  Submission saved successfully to: {out_path}")
# print(submission.head())
